# Data processing v1.1 and v1.2
Changes:
- v1.1 and v1.2: no filtering of 2023-set negatives based on positives of the old set
- v1.2: no minimum length added


In [ ]:

import pandas as pd
import os
from matplotlib import pyplot as plt
from subprocess import DEVNULL, STDOUT, check_call
pos_dataset = '../original_data/enriched.txt'
neg_dataset = '../original_data/depleted.txt'
old_pos_dataset = '../original_data/Pp_resultstable_enriched.txt'
MIN_LEN=50
out_dir = '../processed_data/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)



In [ ]:
def write_fasta_for_clustering(df):
    with open('tmp_cdhit.fasta','w') as write_to:
        for idx, row in df.iterrows():
            print(f'>{int(row.INDEX)}\n{row.SEQ}',file=write_to)

def run_cdhit(cdhit_cutoff):
    # check_call(['cd-hit', '-i tmp_cdhit.fasta', '-o tmp_cdhit.out', f'-c {cdhit_cutoff}', f'-s {cdhit_cutoff}', f'-d 0'], stdout=DEVNULL, stderr=STDOUT, shell=True)
    !echo "Running cdhit with" {cdhit_cutoff}
    !cd-hit -i tmp_cdhit.fasta -o tmp_cdhit.out -c {cdhit_cutoff} -s {cdhit_cutoff} -d 0 &>/dev/null
    
def process_cdhit_outputs():
    results = []
    cluster_number = None
    for line in open('tmp_cdhit.out.clstr'):
        if not line.startswith('>'):
            results.append((int(line[line.index('>')+1:line.index('...')]), cluster_number))
        else:
            cluster_number = int(line.split(' ')[1].strip())
    ret = pd.DataFrame(results, columns=['INDEX', 'CLUSTER'])
    return ret

In [ ]:
def filter_records(main_pos_file, main_neg_file, fc_cutoff, cdhit_cutoff, old_pos_file=None, old_neg_file=None):
    print(f'FC cutoff: {fc_cutoff}, arbitrary clustering cutoff: {cdhit_cutoff}')
    # new data
    df_pos = pd.read_csv(main_pos_file, sep='\t')
    df_pos.drop_duplicates(inplace=True)
    df_neg = pd.read_csv(main_neg_file, sep='\t')
    df_neg.drop_duplicates(inplace=True)

    df_pos = df_pos[df_pos['protein'].str.len()>=MIN_LEN]
    df_neg = df_neg[df_neg['protein'].str.len()>=MIN_LEN]
    df_pos['label'] = 1
    df_neg['label'] = 0
    if 'logFC_P1' in df_pos.columns:
        df_pos = df_pos[(df_pos.logFC_P1>=fc_cutoff) & (df_pos.logFC_P3>=fc_cutoff)]
        df_neg = df_neg[(df_neg.logFC_N1>=fc_cutoff) & (df_neg.logFC_N3>=fc_cutoff)]
    else:
        df_pos = df_pos[(df_pos.logFC_rep3>=fc_cutoff) & (df_pos.logFC_rep4>=fc_cutoff) & (df_pos.logFC_rep5>=fc_cutoff)]
        df_neg = df_neg[(df_neg.logFC_rep3<=-fc_cutoff) & (df_neg.logFC_rep4<=-fc_cutoff) & (df_neg.logFC_rep5<=-fc_cutoff)]

    if old_pos_file and old_neg_file:
        # old data
        old_df_pos = pd.read_csv(old_pos_dataset, sep='\t')
        old_df_pos.drop_duplicates(inplace=True)
        old_df_pos = old_df_pos[old_df_pos['protein'].str.len()>=MIN_LEN]
        old_df_pos['label'] = 1
        old_df_pos = old_df_pos[(old_df_pos.logFC_rep3>=fc_cutoff) & (old_df_pos.logFC_rep4>=fc_cutoff) & (old_df_pos.logFC_rep5>=fc_cutoff)]

        # merge
        old_df_pos = old_df_pos[['protein', 'label']]
        old_df_pos.columns = ['SEQ', 'LABEL']
    
    df_pos = df_pos[['protein', 'label']]
    df_neg = df_neg[['protein', 'label']]
    df = pd.concat([df_pos, df_neg]).reset_index()
    df.columns = ['INDEX', 'SEQ', 'LABEL']

    new_df = df
    # get clusters for new dataset only
    write_fasta_for_clustering(new_df)
    run_cdhit(cdhit_cutoff)
    cluster_info = process_cdhit_outputs()
    new_df = new_df.merge(cluster_info, on='INDEX', how='left')

    # get clusters with inconsistent labels
    unique_labels_per_cluster = new_df.groupby("CLUSTER").LABEL.nunique()
    unique_labels_per_cluster.name = 'unique_labels'
    new_df = new_df.merge(unique_labels_per_cluster, on='CLUSTER', how='left')

    # do analysis on those labels (numbers)
    print(' > numbers on new dataset')
    new_df['source'] = 'new'

    # filter out those clusters
    filtered_df = new_df[new_df.unique_labels==1]
    filtered_df = filtered_df[['SEQ', 'LABEL', 'source']]

    if old_pos_file and old_neg_file:
        print('TMP### adding old enriched data')
        # construct full dataset with enriched from previous set
        old_df_pos['source'] = 'old'
        full_df = pd.concat([filtered_df, old_df_pos])
        full_df.reset_index(inplace=True)
        full_df.columns = ['INDEX', 'SEQ', 'LABEL', 'source']

        # get clusters with inconsistent labels
        write_fasta_for_clustering(full_df)
        run_cdhit(cdhit_cutoff)
        cluster_info = process_cdhit_outputs()
        full_df = full_df.merge(cluster_info, on='INDEX', how='left')
        unique_labels_per_cluster = full_df.groupby("CLUSTER").LABEL.nunique()
        unique_labels_per_cluster.name = 'unique_labels'
        full_df = full_df.merge(unique_labels_per_cluster, on='CLUSTER', how='left')

        # do analysis on those labels (numbers)
        print(' > numbers on new dataset + old enriched')

        # filter out those clusters
        final_df = full_df[full_df.unique_labels==1]
        final_df = final_df[final_df.source == 'new']
    else:
        filtered_df.reset_index(inplace=True)
        filtered_df.columns = ['INDEX', 'SEQ', 'LABEL', 'source']
        final_df = filtered_df

    # get final numbers
    print(' > final numbers')
    print(' - Number of enriched entries: {}'.format(len(final_df[final_df.LABEL==1].drop_duplicates())))
    print(' - Number of depleted entries: {}'.format(len(final_df[final_df.LABEL==0].drop_duplicates())))

    return final_df, new_df

results = []




In [ ]:
# create 1.0 and 5.0 datasets, where there is no overlap between train/valid/test within one, and between train/valid and test splits between the two (when clustering similar sequences at 0.7)
div = [0.7, 0.1, 0.2]
df_at_fc1, _ = filter_records('../data/enriched.txt', '../data/depleted.txt', 1, 0.97)
df_at_fc5, _ = filter_records('../data/enriched.txt', '../data/depleted.txt', 5, 0.97)

df_at_fc5.INDEX = df_at_fc5.INDEX + 1000000

write_fasta_for_clustering(df_at_fc1)

cdhit_cutoff = 0.7
run_cdhit(cdhit_cutoff)
cluster_info = process_cdhit_outputs()
cluster_info.columns = ['INDEX', 'CLUSTER_FC1']
print(cluster_info)
df_at_fc1 = df_at_fc1.merge(cluster_info, on='INDEX', how='left')

write_fasta_for_clustering(df_at_fc5)
run_cdhit(cdhit_cutoff)
cluster_info = process_cdhit_outputs()
cluster_info.columns = ['INDEX', 'CLUSTER_FC5']
df_at_fc5 = df_at_fc5.merge(cluster_info, on='INDEX', how='left')

combined_fc1_fc5_start = pd.concat([df_at_fc1, df_at_fc5])
write_fasta_for_clustering(combined_fc1_fc5_start)
run_cdhit(cdhit_cutoff)
cluster_info = process_cdhit_outputs()
cluster_info.columns = ['INDEX', 'CLUSTER_FC1_FC5']
combined_fc1_fc5_start = combined_fc1_fc5_start.merge(cluster_info, on='INDEX', how='left')


In [ ]:
combined_fc1_fc5 = combined_fc1_fc5_start.copy()
# given a dataframe with three columns CLUSTER_FC1, CLUSTER_FC5, CLUSTER_FC1_FC5. 
# group by CLUSTER_FC1_FC5, and join clusters when entries are in the same CLUSTER_FC1 or CLUSTER_FC5
done = False
clustermania = combined_fc1_fc5.groupby('CLUSTER_FC1_FC5',as_index=False).agg(set)[['CLUSTER_FC1_FC5', 'CLUSTER_FC1', 'CLUSTER_FC5']]
# clustermania.CLUSTER_FC1 = clustermania.CLUSTER_FC1.map(lambda x: {x for x in x if x>=0})
# clustermania.CLUSTER_FC5 = clustermania.CLUSTER_FC5.map(lambda x: {x for x in x if x>=0})
clustermania['FINAL_CLUSTER'] = clustermania.CLUSTER_FC1_FC5

grouped_on_fc1 = combined_fc1_fc5.groupby('CLUSTER_FC1',as_index=False).agg(set)[['CLUSTER_FC1_FC5', 'CLUSTER_FC1', 'CLUSTER_FC5']]
fc1fc5_groups_to_combine = list(grouped_on_fc1[grouped_on_fc1.apply(lambda x: len(x.CLUSTER_FC1_FC5)>1,axis=1)].CLUSTER_FC1_FC5)
for group in fc1fc5_groups_to_combine:
    clustermania.loc[clustermania.FINAL_CLUSTER.isin(group), 'FINAL_CLUSTER'] = list(group)[0]
    
grouped_on_fc5 = combined_fc1_fc5.groupby('CLUSTER_FC5',as_index=False).agg(set)[['CLUSTER_FC1_FC5', 'CLUSTER_FC1', 'CLUSTER_FC5']]
fc1fc5_groups_to_combine = list(grouped_on_fc5[grouped_on_fc5.apply(lambda x: len(x.CLUSTER_FC1_FC5)>1,axis=1)].CLUSTER_FC1_FC5)
for group in fc1fc5_groups_to_combine:
    clustermania.loc[clustermania.FINAL_CLUSTER.isin(group), 'FINAL_CLUSTER'] = list(group)[0]

combined_fc1_fc5 = combined_fc1_fc5.merge(clustermania[['CLUSTER_FC1_FC5', 'FINAL_CLUSTER']], on='CLUSTER_FC1_FC5', how='left')
# fc1_only_with_final_cluster = combined_fc1_fc5[combined_fc1_fc5.INDEX < 1000000]
# fc5_only_with_final_cluster = combined_fc1_fc5[combined_fc1_fc5.INDEX >= 1000000]

combined_fc1_fc5.loc[combined_fc1_fc5.INDEX <  1000000, 'template'] = f'{out_dir}/2023_fc1_{"{}"}.csv'
combined_fc1_fc5.loc[combined_fc1_fc5.INDEX >= 1000000, 'template'] = f'{out_dir}/2023_fc5_{"{}"}.csv'
combined_fc1_fc5 = combined_fc1_fc5[['INDEX', 'SEQ', 'LABEL', 'FINAL_CLUSTER', 'template']]
combined_fc1_fc5.drop_duplicates(inplace=True)

In [ ]:
def divide_clusters(df, cluster_col, div):
    cluster_sizes = df.groupby(cluster_col,as_index=False).agg(len)[['SEQ', 'LABEL', cluster_col]]	
    # divide cluster_sizes in three groups according to a split dictated by div
    # shuffle cluster_sizes first to avoid bias
    cluster_sizes = cluster_sizes.sample(frac=1)
    cluster_sizes['cumsum'] = cluster_sizes.SEQ.cumsum()
    cluster_sizes['cumsum'] = cluster_sizes['cumsum'] / cluster_sizes.SEQ.sum()
    cluster_sizes['set'] = 'train'
    cluster_sizes.loc[cluster_sizes['cumsum'] > div[0], 'set'] = 'valid'
    cluster_sizes.loc[cluster_sizes['cumsum'] > div[0]+div[1], 'set'] = 'test'
    cluster_sizes = cluster_sizes[[cluster_col, 'set']]

    df = df.merge(cluster_sizes, on=cluster_col, how='left')


    for template in df.template.unique():
        sub_df = df[df.template==template]

        ### filter within one sub_df on cdhit clusters (0.97)
        cdhit_cutoff = 0.97
        write_fasta_for_clustering(sub_df)
        run_cdhit(cdhit_cutoff)
        cluster_info = process_cdhit_outputs()
        cluster_info.columns = ['INDEX', 'CLUSTER']
        sub_df = sub_df.merge(cluster_info, on='INDEX', how='left')
        # keep longest sequence per cluster
        sub_df = sub_df.sort_values('SEQ').groupby('CLUSTER', as_index=False).last()
        print(sub_df)
        #######

        sub_df[sub_df.set=='train'][['SEQ', 'LABEL']].to_csv(template.format('train'), index=False)
        sub_df[sub_df.set=='valid'][['SEQ', 'LABEL']].to_csv(template.format('valid'), index=False)
        sub_df[sub_df.set=='test'][['SEQ', 'LABEL']].to_csv(template.format('test'), index=False)
        
    return df

In [ ]:
div = [0.7, 0.1, 0.2]
combined_fc1_fc5_with_sets = divide_clusters(combined_fc1_fc5, 'FINAL_CLUSTER', div)

In [ ]:
combined_fc1_fc5_with_sets

In [ ]:
seqx='DKDDDTTRVDESLNIKVEAEEEKAKSGDETNKEEDEDDEEAEEEEEEEEEEEDEDDDD'
combined_fc1_fc5[combined_fc1_fc5.SEQ.str.contains("DKDDDTTRVDESLNIKVEAEEEKAKSGDETNKEEDEDDEEAEEEEEEEEEEEDEDDDD")]

In [ ]:
### NOW FOR OLD P.P. DATA ###
### THING HERE IS THAT IT NEEDS TO DO THE ABOVE STUFF, BUT ALSO MAKE SURE THERE ARE NO TRAINING / VALIDATION SEQUENCES FROM THE NEW DATASETS IN THE TEST SET ###
old_df_at_fc1, _ = filter_records('../data/Pp_resultstable_enriched.txt', '../data/Pp_resultstable_depleted.txt', 1, 0.97)
old_df_at_fc5, _ = filter_records('../data/Pp_resultstable_enriched.txt', '../data/Pp_resultstable_depleted.txt', 5, 0.97)

old_df_at_fc1.INDEX = old_df_at_fc1.INDEX + 2000000
old_df_at_fc5.INDEX = old_df_at_fc5.INDEX + 3000000

write_fasta_for_clustering(old_df_at_fc1)
cdhit_cutoff = 0.7
run_cdhit(cdhit_cutoff)
cluster_info = process_cdhit_outputs()
cluster_info.columns = ['INDEX', 'CLUSTER_FC1']
old_df_at_fc1 = old_df_at_fc1.merge(cluster_info, on='INDEX', how='left')

write_fasta_for_clustering(old_df_at_fc5)
run_cdhit(cdhit_cutoff)
cluster_info = process_cdhit_outputs()
cluster_info.columns = ['INDEX', 'CLUSTER_FC5']
old_df_at_fc5 = old_df_at_fc5.merge(cluster_info, on='INDEX', how='left')

old_combined_fc1_fc5_start = pd.concat([old_df_at_fc1, old_df_at_fc5, combined_fc1_fc5_with_sets])
write_fasta_for_clustering(old_combined_fc1_fc5_start)
run_cdhit(cdhit_cutoff)
cluster_info = process_cdhit_outputs()
cluster_info.columns = ['INDEX', 'CLUSTER_FC1_FC5_and_old']
old_combined_fc1_fc5_start = old_combined_fc1_fc5_start.merge(cluster_info, on='INDEX', how='left')



In [ ]:
old_combined_fc1_fc5_start

In [ ]:
combined_fc1_fc5_o = old_combined_fc1_fc5_start.copy()[['INDEX','CLUSTER_FC1_FC5_and_old', 'CLUSTER_FC1', 'CLUSTER_FC5', 'set','SEQ','LABEL']]
# given a dataframe with three columns CLUSTER_FC1, CLUSTER_FC5, CLUSTER_FC1_FC5. 
# group by CLUSTER_FC1_FC5, and join clusters when entries are in the same CLUSTER_FC1 or CLUSTER_FC5
done = False
clustermania = combined_fc1_fc5_o.groupby('CLUSTER_FC1_FC5_and_old',as_index=False).agg(set)[['CLUSTER_FC1_FC5_and_old', 'CLUSTER_FC1', 'CLUSTER_FC5', 'set']]


In [ ]:
clustermania

In [ ]:
clustermania['is_in_new_train_or_valid'] = clustermania.set.map(lambda x: 'train' in x or 'valid' in x)
# clustermania.CLUSTER_FC1 = clustermania.CLUSTER_FC1.map(lambda x: {x for x in x if x>=0})
# clustermania.CLUSTER_FC5 = clustermania.CLUSTER_FC5.map(lambda x: {x for x in x if x>=0})
clustermania['FINAL_CLUSTER'] = clustermania.CLUSTER_FC1_FC5_and_old

grouped_on_fc1 = combined_fc1_fc5_o.groupby('CLUSTER_FC1',as_index=False).agg(set)[['CLUSTER_FC1_FC5_and_old', 'CLUSTER_FC1', 'CLUSTER_FC5']]
fc1fc5_groups_to_combine = list(grouped_on_fc1[grouped_on_fc1.apply(lambda x: len(x.CLUSTER_FC1_FC5_and_old)>1,axis=1)].CLUSTER_FC1_FC5_and_old)
for group in fc1fc5_groups_to_combine:
    clustermania.loc[clustermania.FINAL_CLUSTER.isin(group), 'is_in_new_train_or_valid'] = clustermania.is_in_new_train_or_valid[clustermania.FINAL_CLUSTER.isin(group)].any()
    clustermania.loc[clustermania.FINAL_CLUSTER.isin(group), 'FINAL_CLUSTER'] = list(group)[0]
    
grouped_on_fc5 = combined_fc1_fc5_o.groupby('CLUSTER_FC5',as_index=False).agg(set)[['CLUSTER_FC1_FC5_and_old', 'CLUSTER_FC1', 'CLUSTER_FC5']]
fc1fc5_groups_to_combine = list(grouped_on_fc5[grouped_on_fc5.apply(lambda x: len(x.CLUSTER_FC1_FC5_and_old)>1,axis=1)].CLUSTER_FC1_FC5_and_old)
for group in fc1fc5_groups_to_combine:
    clustermania.loc[clustermania.FINAL_CLUSTER.isin(group), 'is_in_new_train_or_valid'] = clustermania.is_in_new_train_or_valid[clustermania.FINAL_CLUSTER.isin(group)].any()
    clustermania.loc[clustermania.FINAL_CLUSTER.isin(group), 'FINAL_CLUSTER'] = list(group)[0]

combined_fc1_fc5_o = combined_fc1_fc5_o.merge(clustermania[['CLUSTER_FC1_FC5_and_old', 'FINAL_CLUSTER', 'is_in_new_train_or_valid']], on='CLUSTER_FC1_FC5_and_old', how='left')


In [ ]:
combined_fc1_fc5_o[combined_fc1_fc5_o.SEQ=="DKDDDTTRVDESLNIKVEAEEEKAKSGDETNKEEDEDDEEAEEEEEEEEEEEDEDDDD"]


In [ ]:

combined_fc1_fc5_o.loc[(2000000 <= combined_fc1_fc5_o.INDEX) & (combined_fc1_fc5_o.INDEX <  3000000), 'template'] = f'{out_dir}/oldPp_fc1_{"{}"}.csv'
combined_fc1_fc5_o.loc[combined_fc1_fc5_o.INDEX >= 3000000, 'template'] = f'{out_dir}/oldPp_fc5_{"{}"}.csv'
combined_fc1_fc5_o = combined_fc1_fc5_o[['INDEX','SEQ', 'LABEL', 'FINAL_CLUSTER', 'template', 'is_in_new_train_or_valid']]
combined_fc1_fc5_o.drop_duplicates(inplace=True)

In [ ]:
combined_fc1_fc5_o

In [ ]:
def divide_clusters_taking_into_account_existing_testsets(df, cluster_col, div):
    cluster_sizes = df.groupby(cluster_col,as_index=False).agg(set)[['SEQ', 'LABEL', cluster_col, 'is_in_new_train_or_valid']]	
    cluster_sizes.SEQ = cluster_sizes.SEQ.map(len)
    cluster_sizes.is_in_new_train_or_valid = cluster_sizes.is_in_new_train_or_valid.map(any)
    # divide cluster_sizes in three groups according to a split dictated by div
    # shuffle cluster_sizes first to avoid bias
    cluster_sizes = cluster_sizes.sample(frac=1)
    cluster_sizes.sort_values('is_in_new_train_or_valid', ascending=False, inplace=True)
    cluster_sizes['cumsum'] = cluster_sizes.SEQ.cumsum()
    cluster_sizes['cumsum'] = cluster_sizes['cumsum'] / cluster_sizes.SEQ.sum()
    cluster_sizes['data_set'] = 'train'
    cluster_sizes.loc[cluster_sizes['cumsum'] > div[0], 'data_set'] = 'valid'
    cluster_sizes.loc[cluster_sizes['cumsum'] > div[0]+div[1], 'data_set'] = 'test'
    cluster_sizes = cluster_sizes[[cluster_col, 'data_set']]

    df = df.merge(cluster_sizes, on=cluster_col, how='left')
    for template in df.template.unique():
        if 'old' not in str(template):
            continue
        sub_df = df[df.template==template]
        print(sub_df)

        ### filter within one sub_df on cdhit clusters (0.97)
        cdhit_cutoff = 0.97
        write_fasta_for_clustering(sub_df)
        run_cdhit(cdhit_cutoff)
        cluster_info = process_cdhit_outputs()
        cluster_info.columns = ['INDEX', 'CLUSTER']
        sub_df = sub_df.merge(cluster_info, on='INDEX', how='left')
        # keep longest sequence per cluster
        sub_df = sub_df.sort_values('SEQ').groupby('CLUSTER', as_index=False).last()
        print(sub_df)
        #######

        sub_df[sub_df.data_set=='train'][['SEQ', 'LABEL']].to_csv(template.format('train'), index=False)
        sub_df[sub_df.data_set=='valid'][['SEQ', 'LABEL']].to_csv(template.format('valid'), index=False)
        sub_df[sub_df.data_set=='test'][['SEQ', 'LABEL']].to_csv(template.format('test'), index=False)
        
    return df

In [ ]:
outputted_df_debugging = divide_clusters_taking_into_account_existing_testsets(combined_fc1_fc5_o, 'FINAL_CLUSTER', div)

In [ ]:
old_naive_enriched_df = pd.read_csv('../data/Pp_resultstable_enriched.txt', sep='\t')
old_naive_depleted_df = pd.read_csv('../data/Pp_resultstable_depleted.txt', sep='\t')
old_naive_enriched_df['LABEL'] = 1
old_naive_depleted_df['LABEL'] = 0
old_naive_df = pd.concat([old_naive_enriched_df, old_naive_depleted_df])
old_naive_df = old_naive_df[['protein', 'LABEL']]
old_naive_df.columns = ['SEQ', 'LABEL']
old_naive_df = old_naive_df[old_naive_df.SEQ.map(len)>=MIN_LEN]
# shuffle and write train/valid/test
old_naive_df = old_naive_df.sample(frac=1)
old_naive_df.iloc[:int(len(old_naive_df)*div[0])][['SEQ', 'LABEL']].to_csv(f'{out_dir}/old_naive_train.csv', index=False)
old_naive_df.iloc[int(len(old_naive_df)*div[0]):int(len(old_naive_df)*(div[0]+div[1]))][['SEQ', 'LABEL']].to_csv(f'{out_dir}/old_naive_valid.csv', index=False)
old_naive_df.iloc[int(len(old_naive_df)*(div[0]+div[1])):][['SEQ', 'LABEL']].to_csv(f'{out_dir}/old_naive_test.csv', index=False)

In [ ]:
old_naive_df.iloc[int(len(old_naive_df)*div[0]):int(len(old_naive_df)*(div[0]+div[1]))][['SEQ', 'LABEL']]

* OLD CODE FROM HERE *